In [3]:
# Importando bibliotecas
import pandas as pd
import os

# Definindo constantes
RAW_PATH = '../data/raw/'
INTERMEDIATE_PATH = '../data/intermediate/'
os.makedirs(INTERMEDIATE_PATH, exist_ok=True)

# Definindo função para carregar dados
def load_data(filename):
    return pd.read_csv(os.path.join(RAW_PATH, filename))

# Definindo mapeamento dos arquivos
tabelas = {
    'customers': 'olist_customers_dataset.csv',
    'geolocation': 'olist_geolocation_dataset.csv',
    'order_items': 'olist_order_items_dataset.csv',
    'order_payments': 'olist_order_payments_dataset.csv',
    'order_reviews': 'olist_order_reviews_dataset.csv',
    'orders': 'olist_orders_dataset.csv',
    'products': 'olist_products_dataset.csv',
    'sellers': 'olist_sellers_dataset.csv',
    'category_translation': 'product_category_name_translation.csv'
}

In [4]:
# Definindo funções de limpeza
def padronizar_colunas(df):
    df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
    return df

def converter_colunas_para_datetime(df, colunas):
    for col in colunas:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

def remover_duplicatas(df):
    return df.drop_duplicates()

def remover_produtos_sem_categoria(df):
    return df.dropna(subset=['product_category_name'])

def adicionar_traducao_categorias(products_df, translation_df):
    return products_df.merge(translation_df, on='product_category_name', how='left')

def verificar_nulos(nome_df, df):
    print(f'🔍 {nome_df.upper()} - Valores nulos:')
    print(df.isnull().sum())
    print()

In [5]:
# Carregando datasets
dataframes = {}
for nome, arquivo in tabelas.items():
    df = pd.read_csv(os.path.join(RAW_PATH, arquivo))
    df = padronizar_colunas(df)
    dataframes[nome] = df

# Obtendo referências para cada dataframe
customers = dataframes['customers']
geolocation = dataframes['geolocation']
order_items = dataframes['order_items']
order_payments = dataframes['order_payments']
order_reviews = dataframes['order_reviews']
orders = dataframes['orders']
products = dataframes['products']
sellers = dataframes['sellers']
category_translation = dataframes['category_translation']

In [6]:
# Aplicando limpeza
# Convertendo para formato datatime
orders = converter_colunas_para_datetime(orders, [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
])

# Removendo produtos sem categoria
products = remover_produtos_sem_categoria(products)

# Adicionando tradução das categorias
products = adicionar_traducao_categorias(products, category_translation)

# Atualizando dict com dataframes atualizados
dataframes['orders'] = orders
dataframes['products'] = products

# Removendo duplicatas
for nome in dataframes:
    dataframes[nome] = remover_duplicatas(dataframes[nome])

# Verificando nulos
for nome, df in dataframes.items():
    verificar_nulos(nome, df)

🔍 CUSTOMERS - Valores nulos:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

🔍 GEOLOCATION - Valores nulos:
geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64

🔍 ORDER_ITEMS - Valores nulos:
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

🔍 ORDER_PAYMENTS - Valores nulos:
order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

🔍 ORDER_REVIEWS - Valores nulos:
review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
r

In [7]:
# Salvando arquivos limpos
for nome, df in dataframes.items():
    caminho = os.path.join(INTERMEDIATE_PATH, f'{nome}.csv')
    df.to_csv(caminho, index=False)
    print(f'✅ {nome}.csv salvo com sucesso!')

✅ customers.csv salvo com sucesso!
✅ geolocation.csv salvo com sucesso!
✅ order_items.csv salvo com sucesso!
✅ order_payments.csv salvo com sucesso!
✅ order_reviews.csv salvo com sucesso!
✅ orders.csv salvo com sucesso!
✅ products.csv salvo com sucesso!
✅ sellers.csv salvo com sucesso!
✅ category_translation.csv salvo com sucesso!
